## Finger Detection Python

##### Install Requirement Pakages

In [1]:
# !pip install -r requirements.txt

##### Import Nesscessary Pakages

In [2]:
from mediapipe import solutions
import cv2
import numpy as np

##### Define Constant

In [3]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54)  # vibrant green

# Define indices for fingertips and key landmarks
FINGER_TIPS = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky

##### Function for check the finger is open

In [4]:
def is_finger_extended(hand_landmarks, tip_idx, lower_idx):
    """
    Check if a finger is extended by comparing the y-coordinate (height)
    of the fingertip and its lower joint.
    """
    # Print the y-coordinates of the fingertip and lower joint
    # print(f"tip_idx.y ({tip_idx}): {hand_landmarks[tip_idx].y}")
    # print(f"lower_idx.y ({lower_idx}): {hand_landmarks[lower_idx].y}")
    
    return hand_landmarks[tip_idx].y < hand_landmarks[lower_idx].y


##### Check hand is outward or inward

In [5]:
def q(hand_landmarks):
    """
    Detect whether the palm is facing inwards or outwards based on landmark positions.
    """
    # Key palm and finger landmarks
    wrist_z = hand_landmarks[0].z
    middle_finger_tip_z = hand_landmarks[12].z
    index_finger_tip_z = hand_landmarks[8].z
    middle_finger_mcp_z = hand_landmarks[9].z

    # Calculate average z-coordinates for palm and fingertips
    palm_z_average = (wrist_z + middle_finger_mcp_z) / 2
    fingertips_z_average = (middle_finger_tip_z + index_finger_tip_z) / 2

    # Detect palm orientation
    if palm_z_average > fingertips_z_average:
        print('Palm Outwards')
        return 1  # Palm facing outwards
    else:
        print('Palm Inwards')
        return 2  # Palm facing inwards


##### Function for count finger

In [6]:
def count_fingers(hand_landmarks, handedness_label):
    """
    Count the number of extended fingers for a detected hand.
    Works with different palm orientations.
    """
    extended_fingers = 0

    # Check non-thumb fingers
    for tip_idx in FINGER_TIPS[1:]:  # Skip the thumb
        lower_joint_idx = tip_idx - 2
        if is_finger_extended(hand_landmarks, tip_idx, lower_joint_idx):
            extended_fingers += 1

    # Improved thumb detection using 3D coordinates
    thumb_tip = hand_landmarks[FINGER_TIPS[0]]
    thumb_ip = hand_landmarks[FINGER_TIPS[0] - 1]  # IP joint
    thumb_mcp = hand_landmarks[FINGER_TIPS[0] - 2]  # MCP joint
    pinky_mcp = hand_landmarks[17]  # Pinky MCP joint

    # Calculate vectors
    vector_thumb = np.array([thumb_tip.x - thumb_ip.x, 
                           thumb_tip.y - thumb_ip.y, 
                           thumb_tip.z - thumb_ip.z])
    vector_palm = np.array([pinky_mcp.x - thumb_mcp.x,
                          pinky_mcp.y - thumb_mcp.y,
                          pinky_mcp.z - thumb_mcp.z])
    
    # Normalize vectors
    vector_thumb = vector_thumb / np.linalg.norm(vector_thumb)
    vector_palm = vector_palm / np.linalg.norm(vector_palm)
    
    # Calculate angle between thumb and palm
    angle = np.arccos(np.clip(np.dot(vector_thumb, vector_palm), -1.0, 1.0))
    angle_degrees = np.degrees(angle)

    # If angle is greater than threshold, thumb is considered extended
    if angle_degrees > 35:  # You may need to adjust this threshold
        extended_fingers += 1

    return extended_fingers

##### Function for draw land mark count finger

In [7]:
def draw_landmarks_and_count_fingers(rgb_image, results):

    annotated_image = np.copy(rgb_image)
    height, width, _ = annotated_image.shape

    total_finger_count = 0  # Initialize total finger count

    for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
        handedness_label = results.multi_handedness[idx].classification[0].label

        # Draw hand landmarks
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())

        # Count extended fingers for the current hand
        finger_count = count_fingers(hand_landmarks.landmark, handedness_label)
        total_finger_count += finger_count  # Add to total finger count

        # Get bounding box for text placement
        x_coordinates = [lm.x for lm in hand_landmarks.landmark]
        y_coordinates = [lm.y for lm in hand_landmarks.landmark]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Display handedness and finger count
        cv2.putText(annotated_image,
                    f"{handedness_label}: {finger_count} fingers",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    # Display total finger count at the top of the image
    total_count_text = f"Total: {total_finger_count}"
    cv2.putText(annotated_image,
                total_count_text,
                (10, 30),  # Top-left corner
                cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, (255, 0, 0), FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

##### Initialize Mediapipe Hands

In [8]:
mp_hands = solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

I0000 00:00:1735829621.212557   14547 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1735829621.216128   14863 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) UHD Graphics (TGL GT1)


##### Using Video for Detection

In [9]:
wCam, hCam = 700, 700
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip frame horizontally for mirror effect
    frame = cv2.flip(frame, 1)

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands
    results = hands.process(rgb_frame)

    # Annotate frame if hands are detected
    if results.multi_hand_landmarks:
        frame = draw_landmarks_and_count_fingers(frame, results)

    # Display annotated frame
    cv2.imshow('Finger Count', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1735829621.268634   14850 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1735829621.303448   14852 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/panha/Desktop/Rupp/Year_4/PP/finger_counter/env/lib/python3.12/site-packages/cv2/qt/plugins"
W0000 00:00:1735829624.434431   14849 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
